# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 5e6 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = True # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "ShooterTraining" # Name of the training environment file.
curriculum_file = "curriculum.json"

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 1024 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 8 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048*32 # How large the experience buffer should be before gradient descent.
learning_rate = 4e-5 # Model learning rate.
hidden_units = 256 # Number of units in hidden layer.
batch_size = 2048 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'ML_Academy' started successfully!


Unity Academy name: ML_Academy
        Number of brains: 1
        Reset Parameters :
		MaxRacketSpeed -> 20.0
		IncMySpeed -> 0.0
Unity brain name: ML_Brain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 9
        Action space type: continuous
        Action space size (per agent): 3
        Memory space size (per agent): 0
        Action descriptions: , , 


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/ppo\model-1650000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-1650000.cptk


Saved Model


INFO:unityagents:
Lesson changed. Now in Lesson 1 : 	MaxRacketSpeed -> 3, IncMySpeed -> 20.2
INFO:unityagents:
Lesson changed. Now in Lesson 2 : 	MaxRacketSpeed -> 4, IncMySpeed -> 20.3
INFO:unityagents:
Lesson changed. Now in Lesson 3 : 	MaxRacketSpeed -> 5, IncMySpeed -> 20.4
INFO:unityagents:
Lesson changed. Now in Lesson 4 : 	MaxRacketSpeed -> 6, IncMySpeed -> 20.5
INFO:unityagents:
Lesson changed. Now in Lesson 5 : 	MaxRacketSpeed -> 7, IncMySpeed -> 20.6
INFO:unityagents:
Lesson changed. Now in Lesson 6 : 	MaxRacketSpeed -> 8, IncMySpeed -> 20.7
INFO:unityagents:
Lesson changed. Now in Lesson 7 : 	MaxRacketSpeed -> 9, IncMySpeed -> 20.8
INFO:unityagents:
Lesson changed. Now in Lesson 8 : 	MaxRacketSpeed -> 10, IncMySpeed -> 20.9
INFO:unityagents:
Lesson changed. Now in Lesson 9 : 	MaxRacketSpeed -> 11, IncMySpeed -> 21
INFO:unityagents:
Lesson changed. Now in Lesson 10 : 	MaxRacketSpeed -> 12, IncMySpeed -> 21.1
INFO:unityagents:
Lesson changed. Now in Lesson 11 : 	MaxRacketSpeed

Step: 1660000. Mean Reward: 2.5821728108241198. Std of Reward: 0.6651860050107011.


INFO:unityagents:
Lesson changed. Now in Lesson 14 : 	MaxRacketSpeed -> 16, IncMySpeed -> 21.5
INFO:unityagents:
Lesson changed. Now in Lesson 15 : 	MaxRacketSpeed -> 17, IncMySpeed -> 21.6
INFO:unityagents:
Lesson changed. Now in Lesson 16 : 	MaxRacketSpeed -> 18, IncMySpeed -> 21.7
INFO:unityagents:
Lesson changed. Now in Lesson 17 : 	MaxRacketSpeed -> 19, IncMySpeed -> 21.8
INFO:unityagents:
Lesson changed. Now in Lesson 18 : 	MaxRacketSpeed -> 20, IncMySpeed -> 21.9
INFO:unityagents:
Lesson changed. Now in Lesson 19 : 	MaxRacketSpeed -> 21, IncMySpeed -> 22.0
INFO:unityagents:
Lesson changed. Now in Lesson 20 : 	MaxRacketSpeed -> 22, IncMySpeed -> 22.1


Step: 1670000. Mean Reward: 1.9332417997698825. Std of Reward: 0.7300503262353528.
Step: 1680000. Mean Reward: 1.9518473284993767. Std of Reward: 0.5502834698436074.
Step: 1690000. Mean Reward: 1.9110525293954717. Std of Reward: 0.5649204090351437.
Step: 1700000. Mean Reward: 1.5957062865885878. Std of Reward: 0.41566624488262954.
Saved Model
Step: 1710000. Mean Reward: 1.7735993385279447. Std of Reward: 0.49008246153229.
Step: 1720000. Mean Reward: 1.9332240939279997. Std of Reward: 0.6340580024309981.
Step: 1730000. Mean Reward: 1.8285801675369462. Std of Reward: 0.5587871635872869.
Step: 1740000. Mean Reward: 1.7512673960956568. Std of Reward: 0.4945090747227337.
Step: 1750000. Mean Reward: 1.865499286525703. Std of Reward: 0.4093517636062059.
Saved Model
Step: 1760000. Mean Reward: 1.9932347064244573. Std of Reward: 0.5783700078510934.
Step: 1770000. Mean Reward: 1.9329897193616028. Std of Reward: 0.5194334657012528.
Step: 1780000. Mean Reward: 1.54519634423591. Std of Reward: 0.71

KeyboardInterrupt: 

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [ ]:
export_graph(model_path, env_name)